In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import  cv2
import matplotlib.pyplot as plt
import radiomics
from radiomics import featureextractor
import mahotas as mh
import numpy as np
import SimpleITK as sitk

2024-08-23 14:23:55.373058: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-23 14:23:55.460342: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-23 14:23:55.484682: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-23 14:23:55.660998: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-23 14:23:57.215051: W tensorflow/compiler/tf2

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


I0000 00:00:1724433839.822007    5441 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1724433840.000024    5441 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1724433840.002463    5441 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


In [3]:
ds = tfds.load('diabetic_retinopathy_detection', split='train', data_dir='/media/guinafelix/PIRISI/dataset')

I0000 00:00:1724433841.973918    5441 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1724433841.976613    5441 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1724433841.978791    5441 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1724433842.130031    5441 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [4]:
ex = ds.take(1)
for example in ex:  # example is `{'image': tf.Tensor, 'label': tf.Tensor}`
  print(list(example.keys()))
  image = example["image"]
  label = example["label"]
  print(label)

['image', 'label', 'name']
tf.Tensor(0, shape=(), dtype=int64)


2024-08-23 14:24:41.979979: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [5]:
# Função auxiliar para filtrar imagens por rótulo
def filter_by_label(example, target_label):
    return tf.equal(example['label'], target_label)

# Função para pegar N exemplos de um dataset
def take_n(dataset, n):
    return dataset.take(n)

# Função que cria um filtro específico para um rótulo
def make_filter_fn(target_label):
    def filter_fn(example):
        return filter_by_label(example, target_label)
    return filter_fn

In [6]:
# Pegar 15 imagens com rótulo 0
label_0_dataset = ds.filter(make_filter_fn(0))
label_0_examples = list(take_n(label_0_dataset, 50))

# Pegar 15 imagens com rótulo 1
label_1_dataset = ds.filter(make_filter_fn(1))
label_1_examples = list(take_n(label_1_dataset, 50))

# Combinar os exemplos em um único dataset para exibição
combined_examples = label_0_examples + label_1_examples

2024-08-23 14:24:43.879462: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [7]:
# Função para exibir as imagens
def display_images(examples):
    plt.figure(figsize=(15, 10))
    for i, example in enumerate(examples):
        image = example['image']
        label = example['label']
        plt.subplot(5, 6, i+1)
        plt.imshow(image.numpy())
        plt.title(f'Label: {label.numpy()}')
        plt.axis('off')
    plt.show()

# Função para exibir uma única imagem
def display_single_image(example):
    image = example['image']
    label = example['label']
    plt.imshow(image.numpy())
    plt.title(f'Label: {label.numpy()}')
    plt.axis('off')
    plt.show()

def display_processed_image(image):
    plt.imshow(image, cmap='gray')  # Defina o mapa de cores como 'gray' se as imagens forem em tons de cinza
    plt.axis('off')
    plt.show()

In [8]:
# Função para aplicar o filtro gaussiano
def apply_gaussian_filter(image):
    return cv2.GaussianBlur(image, (5, 5), 0)

# Função para aplicar equalização adaptativa do histograma
def apply_clahe(image):
    # Converter para espaço de cor YUV
    yuv_img = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
    # Aplicar CLAHE apenas no canal de luminância
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    yuv_img[:,:,0] = clahe.apply(yuv_img[:,:,0])
    # Converter de volta para espaço de cor RGB
    return cv2.cvtColor(yuv_img, cv2.COLOR_YUV2RGB)

# Função para padronizar a imagem conforme Gama et al. 2020
def standardize_image_gama(image):
    # Extrair a camada verde
    green_channel = image[:, :, 1]
    # Aplicar equalização adaptativa do histograma na camada verde
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    standardized_image = clahe.apply(green_channel)
    return standardized_image

# Função para aplicar ambas as metodologias de padronização
def standardize_image(example):
    image = example['image'].numpy()
    # Aplicar o filtro gaussiano
    image = apply_gaussian_filter(image)
    # Aplicar a padronização conforme Gama et al. 2020
    standardized_image = standardize_image_gama(image)
    return standardized_image

# Função para processar o dataset
def preprocess_dataset(dataset):
    standardized_images = []
    for example in dataset:
        standardized_image = standardize_image(example)
        standardized_images.append(standardized_image)
    return standardized_images

In [9]:
standardized_images = preprocess_dataset(combined_examples)

In [10]:
# Função para criar uma máscara simples que cobre parte da imagem
def create_simple_mask(image):
    mask = np.zeros_like(image, dtype=np.uint8)
    h, w = mask.shape[:2]
    # Definir uma região retangular na máscara
    mask[h//4:3*h//4, w//4:3*w//4] = 1
    return mask

# Função para extrair características radiômicas de uma imagem com máscara
def extract_radiomic_features(image):
    # Configurar o extrator de características do PyRadiomics
    params = {"binWidth": 25}  # Parâmetros opcionais para o extrator de características
    extractor = radiomics.featureextractor.RadiomicsFeatureExtractor(**params)

    # Converter a imagem para o formato SimpleITK
    sitk_image = sitk.GetImageFromArray(image)

    # Criar uma máscara simples para a imagem
    mask = create_simple_mask(image)
    sitk_mask = sitk.GetImageFromArray(mask)

    # Extrair características radiômicas
    feature_vector = extractor.execute(sitk_image, sitk_mask)

    return feature_vector

# Lista para armazenar as características radiômicas de todas as imagens
all_feature_vectors = []

# Iterar sobre as imagens padronizadas e extrair características radiômicas
for image in standardized_images:
    feature_vector = extract_radiomic_features(image)
    all_feature_vectors.append(feature_vector)

# Exibir o número de características extraídas para a primeira imagem
print("Número de características extraídas para a primeira imagem:", len(all_feature_vectors[0]))

# Exemplo de como acessar as características radiômicas para a primeira imagem
print("Características radiômicas para a primeira imagem:", all_feature_vectors[0])


INFO:radiomics.featureextractor:No valid config parameter, using defaults: {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True}
INFO:radiomics.featureextractor:Enabled image types: {'Original': {}}
INFO:radiomics.featureextractor:Enabled features: {'firstorder': [], 'glcm': [], 'gldm': [], 'glrlm': [], 'glszm': [], 'ngtdm': [], 'shape': []}
INFO:radiomics.featureextractor:Applying custom setting overrides: {'binWidth': 25}
INFO:radiomics.featureextractor:Calculating features with label: 1
INFO:radiomics.featureextractor:Loading image and mask
Shape features are only available 3D input (for 2D input, use shape2D). Found 2D input
INFO:radiomics.featureextractor:Adding image type "Original" with custom settings: 

Número de características extraídas para a primeira imagem: 115
Características radiômicas para a primeira imagem: OrderedDict([('diagnostics_Versions_PyRadiomics', 'v3.0.1'), ('diagnostics_Versions_Numpy', '1.26.4'), ('diagnostics_Versions_SimpleITK', '2.3.1'), ('diagnostics_Versions_PyWavelet', '1.6.0'), ('diagnostics_Versions_Python', '3.10.12'), ('diagnostics_Configuration_Settings', {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True, 'binWidth': 25}), ('diagnostics_Configuration_EnabledImageTypes', {'Original': {}}), ('diagnostics_Image-original_Hash', '327550199efb0f0fa3027110d7ea873b8e7ae53a'), ('diagnostics_Image-original_Dimensionality', '2D'), ('diagnostics_Image-original_Spacing', (1.0, 1.0)), ('di

In [11]:

# Função para extrair características de textura usando Mahotas
def extract_texture_features(image):
    # Converter a imagem para escala de cinza se necessário
    if image.ndim == 3 and image.shape[2] == 3:
        grey_image = mh.colors.rgb2grey(image)
    else:
        grey_image = image  # Se já estiver em escala de cinza, não é necessário converter
    
    # Extrair características de textura, por exemplo, LBP
    lbp_features = mh.features.lbp(grey_image, radius=3, points=24)
    return lbp_features

# Lista para armazenar as características de textura de todas as imagens
texture_features_list = []

# Iterar sobre as imagens em combined_examples e extrair características de textura
for example in combined_examples:
    image = example['image'].numpy()  # Obter a imagem como uma matriz numpy
    # Extrair características de textura com Mahotas
    texture_features = extract_texture_features(image)
    texture_features_list.append(texture_features)

# Exemplo de como acessar as características de textura para a primeira imagem
lbp_features_first_image = texture_features_list[0]
print("Características de textura (LBP) para a primeira imagem:", lbp_features_first_image)

Características de textura (LBP) para a primeira imagem: [8.670592e+06 2.822880e+05 1.691510e+05 ... 2.806000e+03 1.972820e+05
 3.301100e+05]


In [12]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
import numpy as np
import csv
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Passo 1: Aplicação de PCA nas características de textura
texture_features_matrix = np.vstack(texture_features_list)
n_samples, n_features = texture_features_matrix.shape
n_components = min(n_samples, n_features)
pca = PCA(n_components=n_components)
reduced_texture_features = pca.fit_transform(texture_features_matrix)

# Passo 2: Processa características individuais
reduced_texture_features_list = [pca.transform(features.reshape(1, -1)).flatten() for features in texture_features_list]

# Diagnóstico e correção de dados não numéricos
def convert_to_float(value):
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

for i, features in enumerate(all_feature_vectors):
    cleaned_features = {key: convert_to_float(value) for key, value in features.items()}
    all_feature_vectors[i] = cleaned_features

# Remover entradas com valores None
all_feature_vectors = [{k: v for k, v in features.items() if v is not None} for features in all_feature_vectors]
reduced_texture_features_list = [[convert_to_float(value) for value in features] for features in reduced_texture_features_list]

# Combina características radiômicas e reduzidas
combined_features = []
for radiomic_features, reduced_features in zip(all_feature_vectors, reduced_texture_features_list):
    radiomic_vector = list(radiomic_features.values())
    combined_feature = radiomic_vector + reduced_features
    combined_features.append(combined_feature)

# Verifica comprimentos
lengths = [len(feature) for feature in combined_features]
print("Comprimentos dos vetores em combined_features:", lengths)
if len(set(lengths)) != 1:
    raise ValueError("Inconsistência no comprimento dos vetores em combined_features.")

# Garantir que todos os dados sejam float
combined_features = [[float(x) for x in feature] for feature in combined_features]

# Passo 4: Criação do Arquivo CSV
with open('features.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['feature' + str(i+1) for i in range(len(combined_features[0]))])
    for row in combined_features:
        writer.writerow(row)

# Passo 5: Conversão para NumPy Arrays e Normalização
X = np.array(combined_features, dtype=float)
y = np.array([example['label'].numpy() for example in combined_examples])

# Normalização das características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Construção e Treinamento da Rede Neural
model = Sequential()

# Camada de entrada
model.add(Dense(356, input_dim=X_scaled.shape[1], activation='relu'))

# Camadas ocultas com ReLU e Dropout
for _ in range(6):
    model.add(Dense(83, activation='relu'))
    model.add(Dropout(0.2))

# Camada de saída com ativação Sigmoid
model.add(Dense(1, activation='sigmoid'))

# Compila o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Validação cruzada com K-Fold
kf = KFold(n_splits=10, shuffle=True)
accuracy_scores = []
sensitivity_scores = []
specificity_scores = []

for train_index, val_index in kf.split(X_scaled):
    X_train, X_val = X_scaled[train_index], X_scaled[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    # Treina o modelo
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), verbose=1)
    
    # Faz previsões
    y_pred = (model.predict(X_val) > 0.5).astype(int).flatten()
    
    # Calcula matriz de confusão
    cm = confusion_matrix(y_val, y_pred, labels=[0, 1])
    
    if cm.size == 4:
        tn, fp, fn, tp = cm.ravel()
        
        # Calcula sensibilidade e especificidade
        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    else:
        sensitivity = specificity = 0  # Se não há ambos os rótulos, não é possível calcular as métricas
    
    accuracy_scores.append(model.evaluate(X_val, y_val, verbose=0)[1])
    sensitivity_scores.append(sensitivity)
    specificity_scores.append(specificity)

# Calcula as métricas médias
average_accuracy = np.mean(accuracy_scores)
average_sensitivity = np.mean(sensitivity_scores)
average_specificity = np.mean(specificity_scores)

print(f"Average Accuracy: {average_accuracy:.2f}")
print(f"Average Sensitivity: {average_sensitivity:.2f}")
print(f"Average Specificity: {average_specificity:.2f}")


Comprimentos dos vetores em combined_features: [198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198]


/home/guinafelix/artigo/artigovenv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


I0000 00:00:1724434930.523697    5578 service.cc:146] XLA service 0x7fe4250385f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1724434930.524070    5578 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2024-08-23 14:42:10.685039: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-23 14:42:11.280756: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


1/3 ━━━━━━━━━━━━━━━━━━━━ 9s 5s/step - accuracy: 0.5312 - loss: 0.6822

I0000 00:00:1724434933.793231    5578 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.5117 - loss: 0.6999 - val_accuracy: 0.5000 - val_loss: 0.6878
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4872 - loss: 0.6899 - val_accuracy: 0.5000 - val_loss: 0.6904
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5039 - loss: 0.6873 - val_accuracy: 0.6000 - val_loss: 0.6876
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5156 - loss: 0.6822 - val_accuracy: 0.6000 - val_loss: 0.6863
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5418 - loss: 0.6893 - val_accuracy: 0.6000 - val_loss: 0.6848
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.5122 - loss: 0.6907 - val_accuracy: 0.7000 - val_loss: 0.6783
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6520 - loss: 0.6617 - val_accuracy: 0.8000 - val_loss: 0.6682
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6882 - loss: 0.6447 - val_accuracy: 0.6000 - val_loss: 0.6620
Epoch 9/10
3/